In [6]:
import pandas as pd
import numpy as np

In [4]:
# Replace 'path_to_file' with the actual path to your file
df = pd.read_excel('BatteryStorage_CleanedData.xlsx')

# Display the first few rows of the data
df.head()

,TRADE_DATE,HOUR,INTERVAL,MARKET,RES_TYPE,TYPE,VALUE
0,2023-01-01 00:00:00,1,1,RTD,LESR,EN,666.849
1,2023-01-01 00:00:00,1,2,RTD,LESR,EN,666.849
2,2023-01-01 00:00:00,1,3,RTD,LESR,EN,666.849
3,2023-01-01 00:00:00,1,4,RTD,LESR,EN,666.849
4,2023-01-01 00:00:00,1,5,RTD,LESR,EN,736.668


In [5]:
df

,TRADE_DATE,HOUR,INTERVAL,MARKET,RES_TYPE,TYPE,VALUE
0,2023-01-01 00:00:00,1,1,RTD,LESR,EN,666.849
1,2023-01-01 00:00:00,1,2,RTD,LESR,EN,666.849
2,2023-01-01 00:00:00,1,3,RTD,LESR,EN,666.849
3,2023-01-01 00:00:00,1,4,RTD,LESR,EN,666.849
4,2023-01-01 00:00:00,1,5,RTD,LESR,EN,736.668
...,...,...,...,...,...,...,...
419955,2024-12-31 00:00:00,24,8,RTD,LESR,SOC,9950.242
419956,2024-12-31 00:00:00,24,9,RTD,LESR,SOC,9879.675
419957,2024-12-31 00:00:00,24,10,RTD,LESR,SOC,9815.746
419958,2024-12-31 00:00:00,24,11,RTD,LESR,SOC,9774.369


In [13]:
df_en = df[(df['TYPE'] == 'EN') & (df['INTERVAL'] == 12)]
df_en

,TRADE_DATE,HOUR,INTERVAL,MARKET,RES_TYPE,TYPE,VALUE
11,2023-01-01 00:00:00,1,12,RTD,LESR,EN,41.976
23,2023-01-01 00:00:00,2,12,RTD,LESR,EN,-5.863
35,2023-01-01 00:00:00,3,12,RTD,LESR,EN,-54.224
47,2023-01-01 00:00:00,4,12,RTD,LESR,EN,19.609
59,2023-01-01 00:00:00,5,12,RTD,LESR,EN,8.992
...,...,...,...,...,...,...,...
419623,2024-12-31 00:00:00,20,12,RTD,LESR,EN,2813.940
419635,2024-12-31 00:00:00,21,12,RTD,LESR,EN,1939.382
419647,2024-12-31 00:00:00,22,12,RTD,LESR,EN,1359.847
419659,2024-12-31 00:00:00,23,12,RTD,LESR,EN,925.175


In [14]:
df_soc = (
    df[df['TYPE'] == 'SOC']
    .groupby(['TRADE_DATE', 'HOUR', 'MARKET', 'RES_TYPE', 'TYPE'], as_index=False)['VALUE']
    .mean()
)
df_soc

,TRADE_DATE,HOUR,MARKET,RES_TYPE,TYPE,VALUE
0,2023-01-01,1,RTD,LESR,SOC,6524.477167
1,2023-01-01,2,RTD,LESR,SOC,6460.896417
2,2023-01-01,3,RTD,LESR,SOC,6465.033667
3,2023-01-01,4,RTD,LESR,SOC,6460.216500
4,2023-01-01,5,RTD,LESR,SOC,6428.138083
...,...,...,...,...,...,...
17536,2024-12-31,20,RTD,LESR,SOC,17351.437917
17537,2024-12-31,21,RTD,LESR,SOC,14547.201250
17538,2024-12-31,22,RTD,LESR,SOC,12487.780417
17539,2024-12-31,23,RTD,LESR,SOC,10838.721333


In [16]:
df_en['TRADE_DATE'] = pd.to_datetime(df_en['TRADE_DATE'])
df_soc['TRADE_DATE'] = pd.to_datetime(df_soc['TRADE_DATE'])

en_keys = df_en[['TRADE_DATE', 'HOUR', 'MARKET', 'RES_TYPE']].drop_duplicates()
soc_keys = df_soc[['TRADE_DATE', 'HOUR', 'MARKET', 'RES_TYPE']].drop_duplicates()

# Compare
missing_in_soc = en_keys.merge(soc_keys, on=['TRADE_DATE', 'HOUR', 'MARKET', 'RES_TYPE'], how='left', indicator=True)
missing_in_soc = missing_in_soc[missing_in_soc['_merge'] == 'left_only']

missing_in_en = soc_keys.merge(en_keys, on=['TRADE_DATE', 'HOUR', 'MARKET', 'RES_TYPE'], how='left', indicator=True)
missing_in_en = missing_in_en[missing_in_en['_merge'] == 'left_only']

print("Combinations in EN but not in SOC:", len(missing_in_soc))
print("Combinations in SOC but not in EN:", len(missing_in_en))


Combinations in EN but not in SOC: 0
Combinations in SOC but not in EN: 44


C:\Users\17336\AppData\Local\Temp\ipykernel_14524\391842647.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_en['TRADE_DATE'] = pd.to_datetime(df_en['TRADE_DATE'])


In [24]:
missing_in_en[['TRADE_DATE', 'HOUR', 'MARKET', 'RES_TYPE']].sort_values(['TRADE_DATE', 'HOUR']).head(10)


,TRADE_DATE,HOUR,MARKET,RES_TYPE
263,2023-01-11,24,RTD,LESR
1069,2023-02-14,14,RTD,LESR
1219,2023-02-20,20,RTD,LESR
1597,2023-03-08,14,RTD,LESR
1750,2023-03-14,24,RTD,LESR
2254,2023-04-04,24,RTD,LESR
2758,2023-04-25,24,RTD,LESR
3444,2023-05-24,15,RTD,LESR
3933,2023-06-13,24,RTD,LESR
4627,2023-07-12,22,RTD,LESR


In [25]:
common_keys = pd.merge(
    df_en[['TRADE_DATE', 'HOUR', 'MARKET', 'RES_TYPE']],
    df_soc[['TRADE_DATE', 'HOUR', 'MARKET', 'RES_TYPE']],
    on=['TRADE_DATE', 'HOUR', 'MARKET', 'RES_TYPE']
)

df_en_common = df_en.merge(common_keys, on=['TRADE_DATE', 'HOUR', 'MARKET', 'RES_TYPE'])
df_soc_common = df_soc.merge(common_keys, on=['TRADE_DATE', 'HOUR', 'MARKET', 'RES_TYPE'])


In [31]:
df_en_common = df_en_common.drop(columns=['INTERVAL'])


In [32]:
df_en_common

,TRADE_DATE,HOUR,MARKET,RES_TYPE,TYPE,VALUE
0,2023-01-01,1,RTD,LESR,EN,41.976
1,2023-01-01,2,RTD,LESR,EN,-5.863
2,2023-01-01,3,RTD,LESR,EN,-54.224
3,2023-01-01,4,RTD,LESR,EN,19.609
4,2023-01-01,5,RTD,LESR,EN,8.992
...,...,...,...,...,...,...
17492,2024-12-31,20,RTD,LESR,EN,2813.940
17493,2024-12-31,21,RTD,LESR,EN,1939.382
17494,2024-12-31,22,RTD,LESR,EN,1359.847
17495,2024-12-31,23,RTD,LESR,EN,925.175


In [30]:
df_soc_common

,TRADE_DATE,HOUR,MARKET,RES_TYPE,TYPE,VALUE
0,2023-01-01,1,RTD,LESR,SOC,6524.477167
1,2023-01-01,2,RTD,LESR,SOC,6460.896417
2,2023-01-01,3,RTD,LESR,SOC,6465.033667
3,2023-01-01,4,RTD,LESR,SOC,6460.216500
4,2023-01-01,5,RTD,LESR,SOC,6428.138083
...,...,...,...,...,...,...
17492,2024-12-31,20,RTD,LESR,SOC,17351.437917
17493,2024-12-31,21,RTD,LESR,SOC,14547.201250
17494,2024-12-31,22,RTD,LESR,SOC,12487.780417
17495,2024-12-31,23,RTD,LESR,SOC,10838.721333


In [33]:
df_en_common.to_csv("df_en_common.csv", index=False)
df_soc_common.to_csv("df_soc_common.csv", index=False)